In [ ]:
# Reload the dependencies
%reload_ext autoreload
%autoreload 2

from dataclasses import dataclass
from datetime import datetime
from typing import List
import sys
sys.path.insert(0, "../..")

@dataclass
class Entity:
    id: str
    blurb: str
    embedding: List[float]

@dataclass
class Claim:
    id: str
    subject: str
    object: str
    valid_start: datetime
    valid_end: datetime
    embedding: List[float]

@dataclass
class ClaimEdge:
    id: str
    from_claim_id: str
    to_claim_id: str
    rel_type: str
    valid_start: datetime
    valid_end: datetime

@dataclass
class Facet:
    blurb: str
    embedding: List[float]
    support_claim_ids: List[str]
    embedding: List[float]

@dataclass
class Profile:
    id: str
    blurb: str
    support_claim_ids: List[str]
    embedding: List[float]

